<a href="https://colab.research.google.com/github/ananyaarya02/TicketMind/blob/main/FAISS%2BMetadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 74.2 MB/s eta 0:00:00


In [2]:
import kagglehub
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle


In [3]:
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'multilingual-customer-support-tickets' dataset.
Path to dataset files: /kaggle/input/multilingual-customer-support-tickets


In [4]:
subfolders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

print("Subfolders:", subfolders)
print("Files:", files)

Subfolders: []
Files: ['dataset-tickets-multi-lang-4-20k.csv', 'dataset-tickets-german_normalized_50_5_2.csv', 'dataset-tickets-multi-lang3-4k.csv', 'dataset-tickets-german_normalized.csv', 'aa_dataset-tickets-multi-lang-5-2-50-version.csv']


In [5]:
data="/kaggle/input/multilingual-customer-support-tickets/dataset-tickets-multi-lang-4-20k.csv"
df = pd.read_csv(data)

In [6]:
df.drop(df[df['language'] == 'de'].index, inplace=True)

In [7]:
df=df.drop("language",axis=1)

In [8]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
df['text'] = df['subject'].fillna('') + " " + df['body'].fillna('')

texts = df['text'].tolist()

In [10]:
df = df.reset_index(drop=True)


In [11]:
embeddings = embedder.encode(texts, show_progress_bar=True)


Batches:   0%|          | 0/373 [00:00<?, ?it/s]

In [12]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings).astype('float32'))

In [13]:
faiss.write_index(index, "faiss1.index")


In [14]:
metadata = df[['subject','body','answer','type','queue','priority']].to_dict(orient='records')

In [15]:
with open("metadata2.pkl", "wb") as f:
    pickle.dump(metadata, f)

In [16]:
print("✅ FAISS + metadata saved")

✅ FAISS + metadata saved
